In [9]:
import datetime as dt
from enum import IntEnum, StrEnum
from pathlib import Path

from pydantic import BaseModel, EmailStr, Field, HttpUrl
from pydantic_ai import Agent

In [2]:
class EducationLevel(IntEnum):
    HIGH_SCHOOL = 1
    ASSOCIATES = 2
    BACHELORS = 3
    MASTERS = 4
    PHD = 5
    OTHER = 0


class EmploymentType(StrEnum):
    FULL_TIME = "full_time"
    PART_TIME = "part_time"
    CONTRACT = "contract"
    INTERNSHIP = "internship"
    FREELANCE = "freelance"
    OTHER = "other"


class Skill(BaseModel):
    name: str
    level: int | None = Field(None, ge=1, le=10)
    years_experience: float | None = None


class Education(BaseModel):
    institution: str
    degree: str = ""
    level: EducationLevel
    field_of_study: str = ""
    gpa: float | None = Field(None, ge=0, le=4.0)
    description: str = ""
    start_date: dt.date | None = None
    end_date: dt.date | None = None


class WorkExperience(BaseModel):
    company: str
    title: str
    employment_type: EmploymentType
    location: str = ""
    is_current: bool = False
    description: str
    achievements: list[str] = Field(default_factory=list)
    start_date: dt.date | None = None
    end_date: dt.date | None = None


class Project(BaseModel):
    name: str
    description: str
    duties: list[str] = Field(default_factory=list)
    company: str = Field(
        "",
        description="The candidate may or may not have been an employee of this company, but the project was done for or with this company",
    )
    links: list[HttpUrl] = Field(default_factory=list)


class ContactInfo(BaseModel):
    email: EmailStr
    phone: str = ""
    location: str = ""
    linkedin: HttpUrl | None = None
    github: HttpUrl | None = None
    portfolio: HttpUrl | None = None


class Resume(BaseModel):
    name: str
    contact_info: ContactInfo
    years_of_experience: float

    summary: str

    work_experience: list[WorkExperience] = Field(default_factory=list)
    education: list[Education] = Field(default_factory=list)
    skills: list[Skill] = Field(default_factory=list)

    certifications: list[str] = Field(default_factory=list)
    languages: list[str] = Field(default_factory=list)
    projects: list[Project] = Field(default_factory=list)

    last_updated: dt.date = Field(default_factory=dt.date.today)
    availability_date: dt.date | None = None


In [3]:
IDEAL_CANDIDATE_PROMPT = """
You are a technical recruiter with expertise in creating candidate profiles. Your task is to create an ideal candidate profile in a specific format based on the job description provided.

Important Guidelines:
1. Use the job description to fill in all relevant fields
2. For fields not explicitly mentioned in the job description:
   - Fill in reasonable values if they can be confidently inferred from the context
   - Example: If job says "senior position" but no years specified, assume 7+ years
   - Example: If location mentions "hybrid in NYC", include NYC in location field
   - Example: If role is about python web development but does not mention FastAPI, add FastAPI to skills
3. Use industry standards and context to fill gaps:
   - Match skills with typical years of experience for the level
   - Include relevant certifications common in the field
   - Add typical projects someone in this role would have
4. Leave fields empty (use default values) if they cannot be reasonably inferred
   - For contact info like linkedin/github/portfolio, only include if specified in job description
5. Ensure all dates are logical and recent
6. Include achievements that would be impressive for this role
""".strip()


In [4]:
job_desc = """
Cybersecurity Engineer
Location: Boston, MA
Type: Full-time

About the Role:
We're seeking a mid-level Cybersecurity Engineer to join our Information Security team. The ideal candidate will help design, implement, and maintain security solutions while responding to security incidents and vulnerabilities.

Required Qualifications:
- Bachelor's degree in Computer Science, Cybersecurity, or related field
- 3-5 years of experience in information security
- Strong knowledge of network security, encryption, and security protocols
- Experience with SIEM tools (Splunk, ELK Stack)
- Proficiency in Python or other scripting languages
- Experience with vulnerability assessment tools
- CompTIA Security+ certification

Key Responsibilities:
- Monitor and respond to security incidents
- Perform vulnerability assessments and penetration testing
- Implement and maintain security tools and infrastructure
- Develop and update security policies and procedures
- Conduct security awareness training
- Collaborate with IT teams on security implementations

Preferred Qualifications:
- CISSP or CEH certification
- Experience with cloud security (AWS/Azure)
- Knowledge of NIST frameworks
- Experience with threat hunting and incident response
""".strip()


In [11]:
async def create_ideal_candidate(job_desc: str, path: Path | str = "") -> Resume:
    ideal_candidate_agent = Agent(
        model="openai:gpt-4o-mini", result_type=Resume, system_prompt=IDEAL_CANDIDATE_PROMPT
    )
    ideal_candidate = await ideal_candidate_agent.run(user_prompt=job_desc)
    if path:
        Path(path).write_text(ideal_candidate.data.model_dump_json())
    return ideal_candidate.data